--- Day 9: Rope Bridge ---


This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where not to step.

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):

....
.TH.
....

....
.H..
..T.
....

...
.H. (H covers T)
...
If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:

.....    .....    .....
.TH.. -> .T.H. -> ..TH.
.....    .....    .....

...    ...    ...
.T.    .T.    ...
.H. -> ... -> .T.
...    .H.    .H.
...    ...    ...
Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:

.....    .....    .....
.....    ..H..    ..H..
..H.. -> ..... -> ..T..
.T...    .T...    .....
.....    .....    .....

.....    .....    .....
.....    .....    .....
..H.. -> ...H. -> ..TH.
.T...    .T...    .....
.....    .....    .....
You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:

R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

== Initial State ==

......
......
......
......
H.....  (H covers T, s)

== R 4 ==

......
......
......
......
TH....  (T covers s)

......
......
......
......
sTH...

......
......
......
......
s.TH..

......
......
......
......
s..TH.

== U 4 ==

......
......
......
....H.
s..T..

......
......
....H.
....T.
s.....

......
....H.
....T.
......
s.....

....H.
....T.
......
......
s.....

== L 3 ==

...H..
....T.
......
......
s.....

..HT..
......
......
......
s.....

.HT...
......
......
......
s.....

== D 1 ==

..T...
.H....
......
......
s.....

== R 4 ==

..T...
..H...
......
......
s.....

..T...
...H..
......
......
s.....

......
...TH.
......
......
s.....

......
....TH
......
......
s.....

== D 1 ==

......
....T.
.....H
......
s.....

== L 5 ==

......
....T.
....H.
......
s.....

......
....T.
...H..
......
s.....

......
......
..HT..
......
s.....

......
......
.HT...
......
s.....

......
......
HT....
......
s.....

== R 2 ==

......
......
.H....  (H covers T)
......
s.....

......
......
.TH...
......
s.....
After simulating the rope, you can count up all of the positions the tail visited at least once. In this diagram, s again marks the starting position (which the tail also visited) and # marks other positions the tail visited:

..##..
...##.
.####.
....#.
s###..
So, there are 13 positions the tail visited at least once.

Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?

In [38]:
import numpy as np
from loguru import logger

with open('../data/day9.txt') as f:
    input_data = [(line.split()[0], int(line.split()[1])) for line in f]

In [63]:
class WalkGrid:
    def __init__(self):
        self.h_coords = [(0,0)]
        self.t_coords = [(0,0)]
                
    def move(self, direction, distance):
        for i in range(distance):
            if direction == 'R':
                self.move_right()
            elif direction == 'L':
                self.move_left()
            elif direction == 'U':
                self.move_up()
            elif direction == 'D':
                self.move_down()
            else:
                logger.error(f'Direction {direction} unknown')
                
    def move_right(self):
        h_x, h_y = self.h_coords[-1]
        new_h_coord = (h_x + 1, h_y)
        self.h_coords.append(new_h_coord)
        old_t_coord = self.t_coords[-1]
        # checks if the x or y distance of the new coordinate is larger then one to see if tail move to old head
        if (abs(new_h_coord[0] - old_t_coord[0]) > 1) or (abs(new_h_coord[1] - old_t_coord[1]) > 1):    
            self.t_coords.append(self.h_coords[-2])
    
    def move_left(self):
        h_x, h_y = self.h_coords[-1]
        new_h_coord = (h_x - 1, h_y)
        self.h_coords.append(new_h_coord)
        old_t_coord = self.t_coords[-1]
        # checks if the x or y distance of the new coordinate is larger then one to see if tail move to old head
        if (abs(new_h_coord[0] - old_t_coord[0]) > 1) or (abs(new_h_coord[1] - old_t_coord[1]) > 1):    
            self.t_coords.append(self.h_coords[-2])
    
    def move_up(self):
        h_x, h_y = self.h_coords[-1]
        new_h_coord = (h_x , h_y + 1)
        self.h_coords.append(new_h_coord)
        old_t_coord = self.t_coords[-1]
        # checks if the x or y distance of the new coordinate is larger then one to see if tail move to old head
        if (abs(new_h_coord[0] - old_t_coord[0]) > 1) or (abs(new_h_coord[1] - old_t_coord[1]) > 1):    
            self.t_coords.append(self.h_coords[-2])
        
    def move_down(self):
        h_x, h_y = self.h_coords[-1]
        new_h_coord = (h_x, h_y - 1)
        self.h_coords.append(new_h_coord)
        old_t_coord = self.t_coords[-1]
        # checks if the x or y distance of the new coordinate is larger then one to see if tail move to old head
        if (abs(new_h_coord[0] - old_t_coord[0]) > 1) or (abs(new_h_coord[1] - old_t_coord[1]) > 1):    
            self.t_coords.append(self.h_coords[-2])

            
walker = WalkGrid()        

for direction, distance in input_data:
    walker.move(direction, distance)

In [64]:
len(set(walker.t_coords))

6498

--- Part Two ---


A rope snaps! Suddenly, the river is getting a lot closer than you remember. The bridge is still there, but some of the ropes that broke are now whipping toward you as you fall through the air!

The ropes are moving too quickly to grab; you only have a few seconds to choose how to arch your body to avoid being hit. Fortunately, your simulation can be extended to support longer ropes.

Rather than two knots, you now must simulate a rope consisting of ten knots. One knot is still the head of the rope and moves according to the series of motions. Each knot further down the rope follows the knot in front of it using the same rules as before.

Using the same series of motions as the above example, but with the knots marked H, 1, 2, ..., 9, the motions now occur as follows:

In [99]:
class WalkGrid_n_knots:
    def __init__(self, knots):
        self.coords = list()
        self.n_knots = knots
        self.add_starting_positions()
        
    def add_starting_positions(self):
        for i in range(self.n_knots):
            coord_list = list()
            coord_list.append((0,0))
            self.coords.append(coord_list)
                
    def move(self, direction, distance):
        for i in range(distance):
            if direction == 'R':
                self.move_right()
            elif direction == 'L':
                self.move_left()
            elif direction == 'U':
                self.move_up()
            elif direction == 'D':
                self.move_down()
            else:
                logger.error(f'Direction {direction} unknown')
                
    def move_right(self):
        h_x, h_y = self.coords[0][-1]
        new_h_coord = (h_x + 1, h_y)
        self.coords[0].append(new_h_coord)
        for i in range(1, self.n_knots):
            top_neighbour_coords = self.coords[i-1][-1]
            current_knot_coords = self.coords[i][-1]
            if (abs(top_neighbour_coords[0] - current_knot_coords[0]) > 1) or (abs(top_neighbour_coords[1] - current_knot_coords[1]) > 1):
                self.coords[i].append(self.coords[i-1][-2])
            
    def move_left(self):
        h_x, h_y = self.coords[0][-1]
        new_h_coord = (h_x - 1, h_y)
        self.coords[0].append(new_h_coord)
        for i in range(1, self.n_knots):
            top_neighbour_coords = self.coords[i-1][-1]
            current_knot_coords = self.coords[i][-1]
            if (abs(top_neighbour_coords[0] - current_knot_coords[0]) > 1) or (abs(top_neighbour_coords[1] - current_knot_coords[1]) > 1):
                self.coords[i].append(self.coords[i-1][-2])
    
    def move_up(self):
        h_x, h_y = self.coords[0][-1]
        new_h_coord = (h_x , h_y + 1)
        self.coords[0].append(new_h_coord)
        for i in range(1, self.n_knots):
            top_neighbour_coords = self.coords[i-1][-1]
            current_knot_coords = self.coords[i][-1]
            if (abs(top_neighbour_coords[0] - current_knot_coords[0]) > 1) or (abs(top_neighbour_coords[1] - current_knot_coords[1]) > 1):
                self.coords[i].append(self.coords[i-1][-2])
        
    def move_down(self):
        h_x, h_y = self.coords[0][-1]
        new_h_coord = (h_x, h_y - 1)
        self.coords[0].append(new_h_coord)
        for i in range(1, self.n_knots):
            top_neighbour_coords = self.coords[i-1][-1]
            current_knot_coords = self.coords[i][-1]
            if (abs(top_neighbour_coords[0] - current_knot_coords[0]) > 1) or (abs(top_neighbour_coords[1] - current_knot_coords[1]) > 1):
                self.coords[i].append(self.coords[i-1][-2])



In [100]:
walker = WalkGrid_n_knots(knots=10)        

for direction, distance in input_data:
    walker.move(direction, distance)

In [101]:
for i in range(10):
    print(len(set(walker.coords[i])))

7345
6498
6215
5957
5756
5593
5464
5355
5276
5209


In [75]:
a

[[(0, 0), (1, 0)], [(0, 0)]]